In [ ]:
import time
import pandas as pd
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, Resize

from core.architecture.datasets.prediction_datasets import PredictionFolderDataset

dataset_path = '/media/n31v/data/datasets/Land-Use_Scene_Classification/images'
transform = Compose([ToTensor(), Resize((256, 256))])
prediction_ds = PredictionFolderDataset(dataset_path, transform=transform)
prediction_dl = DataLoader(prediction_ds, batch_size=32, num_workers=8)
inference_time = pd.DataFrame()

In [ ]:
from torchvision.models import resnet18
from core.architecture.experiment.nn_experimenter import ClassificationExperimenter

i_time = []
for i in range(1, 6):
    exp = ClassificationExperimenter(resnet18(num_classes=21))
    exp.load_model(f'../models/Land-Use_Scene_Classification/ResNet18({i}fold)/train')
    start = time.time()
    exp.predict(prediction_dl)
    i_time.append(time.time()-start)
inference_time['Baseline'] = i_time

In [ ]:
from core.operation.optimization.structure_optimization import SFPOptimization

sfp_optimization = SFPOptimization(pruning_ratio=0.5)
i_time = []
for i in range(1, 6):
    exp = ClassificationExperimenter(resnet18(num_classes=21))
    sfp_optimization.load_model(
        exp=exp,
        state_dict_path=f'../models/Land-Use_Scene_Classification/ResNet18({i}fold)_SFP_P-0.50/pruned.sd.pt'
    )
    start = time.time()
    exp.predict(prediction_dl)
    i_time.append(time.time()-start)
inference_time['SFP 50%'] = i_time

In [ ]:
from core.operation.optimization.structure_optimization import SVDOptimization

energy_thresholds = [
    0.1, 0.3, 0.5, 0.7, 0.9,
    0.91, 0.93, 0.95, 0.97, 0.99,
    0.995, 0.999,
    0.9999, 1
]
svd_optimization = SVDOptimization(energy_thresholds=energy_thresholds, decomposing_mode='spatial')
for e in energy_thresholds:
    i_time = []
    for i in range(1, 6):
        exp = ClassificationExperimenter(resnet18(num_classes=21))
        svd_optimization.load_model(
            exp=exp,
            state_dict_path=f'../models/Land-Use_Scene_Classification/ResNet18({i}fold)_SVD_spatial_O-100.0_H-0.001000/e_{e}.sd.pt'
        )
        start = time.time()
        exp.predict(prediction_dl)
        i_time.append(time.time()-start)
    inference_time[f'SVD e={e}'] = i_time

In [ ]:
svd_optimization = SVDOptimization(energy_thresholds=energy_thresholds)
for e in energy_thresholds:
    i_time = []
    for i in range(1, 6):
        exp = ClassificationExperimenter(resnet18(num_classes=21))
        svd_optimization.load_model(
            exp=exp,
            state_dict_path=f'../models/Land-Use_Scene_Classification/ResNet18({i}fold)_SVD_channel_O-100.0_H-0.001000/e_{e}.sd.pt'
        )
        start = time.time()
        exp.predict(prediction_dl)
        i_time.append(time.time()-start)
    inference_time[f'SVD e={e}'] = i_time

In [ ]:
inference_time.mean()